# Mean Variance Frontier w/w.o Short Sales Constraints

This code below calculates mean variances frontiers (a) when there are no restrictions on the portfolio weights and (b) when we impose the restriction that no weights can be negative.

The latter case requires a numerical minimization routine (the code uses quadprog), which comes from the packages MathProgBase and Ipopt. You may have to install these to run the code (see below for instructions).

# Load Packages

In [1]:
using MathProgBase, Ipopt
#using Dates       #Julia 0.7

include("printmat.jl")    

printmatDate

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# The Inputs to the MV Calculations

In [3]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;          #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printlnPs(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030


## From Ch2 Notebook (MV Frontier without portfolio restrictions)

In [4]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF
    n    = length(μ)
    oneV = ones(n) 
    Σ_1  = inv(Σ)
    A    = μ'Σ_1*μ
    B    = μ'Σ_1*oneV
    C    = oneV'Σ_1*oneV
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

MVCalc (generic function with 1 method)

In [5]:
mustar = linspace(0.04,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRp[i] = MVCalc(mustar[i],μ,Σ)[1]
end    

In [6]:
plot(StdRp*100,mustar*100,color=:red,linewidth=2,legend=false)
plot!(xlim=(0,15),ylim=(0,15))
title!("Mean variance frontier (no portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean variance frontier (no portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 437.294,272.447 430.316,270.037 423.405,267.628 416.565,265.218 409.801,262.809 403.116,260.399 396.516,257.99 390.004,255.58 383.586,253.171 377.268,250.761 
 371.055,248.352 364.953,245.942 358.969,243.533 353.11,241.123 347.383,238.714 341.796,236.304 336.356,233.895 331.072,231.485 325.953,229.076 321.008,226.666 
 316.246,224.257 311.678,221.847 307.313,219.438 303.162,217.028 299.235,214.619 295.543,212.209 292.095,209.8 288.904,207.39 285.978,204.981 283.328,202.571 
 280.962,200.162 278.89,197.752 277.119,195.343 275.656,192.933 274.507,190.524 273.677,188.114 273.169,185.705 272.985,183.295 273.127,180.886 273.593,178.476 
 274.382,176.067 275.49,173.657 276.913,171.248 278.645,168.838 280.679,166.429 283.007,164.019 285.621,161.61 288.512,159.2 291.67,156.791 295.085,154.381 
 298.747,151.971 302.644,149.562 306.767,147.152 311.105,144.743 315.648,142.333 320.385,139.924 325.307,137.514 330.404,135.105 335.667,132.695 341.088,130.286 
 346.657,127.876 352.366,125.467 358.209,123.057 364.177,120.648 370.264,118.238 376.463,115.829 382.768,113.419 389.173,111.01 395.673,108.6 402.262,106.191 
 408.937,103.781 415.691,101.372 422.521,98.9623 429.423,96.5528 436.393,94.1433 443.427,91.7338 450.523,89.3243 457.676,86.9148 464.884,84.5053 472.145,82.0957 
 479.455,79.6862 486.812,77.2767 494.214,74.8672 501.659,72.4577 509.144,70.0482 516.668,67.6387 524.228,65.2292 531.824,62.8197 539.453,60.4102 547.115,58.0007 
 554.806,55.5911 562.527,53.1816 570.276,50.7721 578.051,48.3626 585.852,45.9531 593.678,43.5436 601.527,41.1341 609.398,38.7246 617.291,36.3151 625.204,33.9056 
 633.137,31.4961 
 "/>

## MV Frontier when Short Sales are Not Allowed

The code below solves (numerically) the following minimization problem 

$\min \text{Var}(R_p) \> \text{ s.t. } \>  \text{E}R_p = \mu^*$,
 
and where we require $w_i\ge 0$
 
The code below calls on
```
quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
```

This means that we minimize
    $0'w + w' \Sigma w/2$ subject to the restrictions $A w = b$ and $0 \leq w \leq 1$. The equality constraints ($A w = b$) are used to impose $1'w=1$ and $\mu'w = \mu^*$ (se the code below).

In [7]:
function MeanVarNoSSPs(mu,Sigma,mustar)   #MV with no short-sales, numerical minimization
    (w_p,StdRp) = (NaN,NaN)     #default result (if no feasible solution)        
    N = length(mu)
    if minimum(mu) <= mustar <= maximum(mu)  #try only if feasible
        A   = [ones(1,N);mu']                #1'w=1, mu'w = mustar
        b   = [1;mustar]
        Sol = quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
        if Sol.status == :Optimal
            w_p = Sol.sol
            StdRp = sqrt(w_p'Sigma*w_p)
        end    
    end    
    return StdRp,w_p
end

MeanVarNoSSPs (generic function with 1 method)

In [8]:
Std_no_ss = fill(NaN,length(mustar))
for i = 1:length(mustar)
    Std_no_ss[i] = MeanVarNoSSPs(μ,Σ,mustar[i])[1]     #[1] to get first output
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [9]:
plot(StdRp*100,mustar*100,color=:red,linewidth=2,label="Risky only")
plot!(Std_no_ss*100,mustar*100,color=:green,line=(:dash,2),label="Risky only (no ss) ")
plot!(xlim=(0,15),ylim=(0,15))
title!("Mean variance frontier (no portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean variance frontier (no portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 437.294,272.447 430.316,270.037 423.405,267.628 416.565,265.218 409.801,262.809 403.116,260.399 396.516,257.99 390.004,255.58 383.586,253.171 377.268,250.761 
 371.055,248.352 364.953,245.942 358.969,243.533 353.11,241.123 347.383,238.714 341.796,236.304 336.356,233.895 331.072,231.485 325.953,229.076 321.008,226.666 
 316.246,224.257 311.678,221.847 307.313,219.438 303.162,217.028 299.235,214.619 295.543,212.209 292.095,209.8 288.904,207.39 285.978,204.981 283.328,202.571 
 280.962,200.162 278.89,197.752 277.119,195.343 275.656,192.933 274.507,190.524 273.677,188.114 273.169,185.705 272.985,183.295 273.127,180.886 273.593,178.476 
 274.382,176.067 275.49,173.657 276.913,171.248 278.645,168.838 280.679,166.429 283.007,164.019 285.621,161.61 288.512,159.2 291.67,156.791 295.085,154.381 
 298.747,151.971 302.644,149.562 306.767,147.152 311.105,144.743 315.648,142.333 320.385,139.924 325.307,137.514 330.404,135.105 335.667,132.695 341.088,130.286 
 346.657,127.876 352.366,125.467 358.209,123.057 364.177,120.648 370.264,118.238 376.463,115.829 382.768,113.419 389.173,111.01 395.673,108.6 402.262,106.191 
 408.937,103.781 415.691,101.372 422.521,98.9623 429.423,96.5528 436.393,94.1433 443.427,91.7338 450.523,89.3243 457.676,86.9148 464.884,84.5053 472.145,82.0957 
 479.455,79.6862 486.812,77.2767 494.214,74.8672 501.659,72.4577 509.144,70.0482 516.668,67.6387 524.228,65.2292 531.824,62.8197 539.453,60.4102 547.115,58.0007 
 554.806,55.5911 562.527,53.1816 570.276,50.7721 578.051,48.3626 585.852,45.9531 593.678,43.5436 601.527,41.1341 609.398,38.7246 617.291,36.3151 625.204,33.9056 
 633.137,31.4961 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 393.031,226.666 382.573,224.257 372.381,221.847 362.481,219.438 352.901,217.028 343.67,214.619 334.824,212.209 326.398,209.8 318.43,207.39 310.963,204.981 
 304.038,202.571 297.701,200.162 291.998,197.752 286.971,195.343 282.666,192.933 279.12,190.524 276.37,188.114 274.444,185.705 273.362,183.295 273.137,180.886 
 273.593,178.476 274.382,176.067 275.49,173.657 276.913,171.248 278.645,168.838 280.679,166.429 283.007,164.019 285.621,161.61 288.512,159.2 291.67,156.791 
 295.085,154.381 298.747,151.971 302.644,149.562 306.767,147.152 311.105,144.743 315.648,142.333 320.385,139.924 325.492,137.514 332.379,135.105 341.22,132.695 
 351.846,130.286 364.079,127.876 377.742,125.467 392.667,123.057 408.698,120.648 425.695,118.238 443.536,115.829 462.112,113.419 481.33,111.01 501.107,108.6 
 
 "/>
 
 
 
 
 Risky only 
 
 
 
 Risky only (no ss)